In [16]:
%reload_ext autoreload
%autoreload 2

! [ -L /datasets ] && rm -f /datasets
! ln -s /data/datasets/ /datasets

from k12libs.utils.nb_easy import k12ai_set_notebook

k12ai_set_notebook(cellw=95)

## 需掌握知识点

KNN, 决策树，随即森林，集成学习等算法原理介绍

In [59]:
%reset -f

import numpy as np
from pyr.app.k12ai import EasyaiClassifier, EasyaiTrainer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

class CustomClassifier(EasyaiClassifier):
    
    def prepare_dataset(self):
        """
        1. titanic:
            features:
                'Pclass':   船舱等级
                'Sex':      性别
                'Age':      年龄
                'SibSp':    兄弟姐妹一起上船 (可能相互让救生圈) 
                'Parch':    父母和小孩 (老人孩子优先)
                'Fare':     费用
                'Embarked': 上船的地方 (地位)
            
            target:
                'Survived': 存活 (目标对象) 1: 存活 0: 死亡
            
        eg: [(2.0, 'Pclass'), (1.0, 'Sex'), (22.0, 'Age'), (1.0, 'SibSp'), (1.0, 'Parch'), (29.0, 'Fare'), (0.0, 'Embarked')]
        """
        return self.load_titanic()
    
    def build_model(self):
        """
        KNN:
        """
        return KNeighborsClassifier(
            n_neighbors = 5,        # [O] 近邻的个数
            weights='uniform',      # [O] 权重 1. uniform: 无论近邻距离多远, 权重一样 2. 'distance': 距离越近权重越大
            algorithm = 'auto',     # [O] 构建算法 1. audo: 自动选择 2. ball_tree(球树) 3. kd_tree(KD树) 4. brute(蛮力实现)
            leaf_size = 30,         # [O] 建子树的叶子节点数量的阈值(algorithm==brute时无效)
            metric = 'minkowski',   # [O] 距离函数 1. euclidean(欧式距离) 2. manhattan(曼哈顿距离) 3. chebyshev(切比雪夫距离) 4. minkowski(闵可夫斯基距离)
            p = 2                   # [O] 当metric='minkowski'时有效, 作为距离函数的参数                  
        )
    
    def configure_estimator(self):
        return {
            'test_size': 0.3,       # [O] 数据样本分割比例
            'scaler_mode': 'zscore' # [O] 数据规范化模式 1. zscore: 采用Z-Score规范化 2. minmax: 采用Min-Max规范化
        }
    
    def fit_end(self, preds, trues):
        print('正确率:')
        print(accuracy_score(preds, trues))
        print('混淆矩阵:')
        print(confusion_matrix(preds, trues))

    def predict_end(self, preds):
        print('预测结果:')
        print(preds)
    
    
trainer = EasyaiTrainer(framework='ml') # [M] 必须指定为"ml"

model = CustomClassifier()

# 训练
trainer.fit(model)

# 预测
features = np.array([
    [2.0, 1.0, 22, 1.0, 1.0, 29.0, 0.0], # 存活
    [9.0, 0.0, 22, 0.0, 0.0, 9.0, 0.0],  # 死亡
])

trainer.predict(model, features=features)

正确率:
0.7873134328358209
混淆矩阵:
[[136  36]
 [ 21  75]]
预测结果:
[1 0]
